In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import requests
import re
import json
from pymongo import MongoClient
from odo import odo

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

#import os
#chromedriver = "/home/allisonschlissel/notebooks/chromedriver"
#os.environ["webdriver.chrome.driver"] = chromedriver



#driver = webdriver.Chrome(chromedriver)
#driver.get("https://mail.google.com")

In [ ]:
lst = ['http://www.metmuseum.org/art/collection#!?material=Paintings&showOnly=withImage&offset=' + str(x) + '&pageSize=0&sortBy=Relevance&sortOrder=asc&perPage=20' for x in range(0, 12017, 20)]
for i in lst:
    print i

In [ ]:
my_links = []

soup = BeautifulSoup(html, 'html.parser')

for figure in soup.find_all('figure',class_="card__standard-image"):
    fig = figure.find('a')['href']
    if '/art/collection/search/' in fig:
        url = 'http://www.metmuseum.org' + fig
        my_links.append(url)
        print url

In [ ]:
soup_objs = []

for i in my_links:
    response = requests.get(i)
    page = response.text
    link_soup = BeautifulSoup(page)
    soup_objs.append(link_soup)
    
response.status_code

In [ ]:
for soup in soup_objs:
    title = soup.find('h1', class_='collection-details__object-title')
    artist = soup.find('div',class_='collection-details__tombstone')
    gallery = soup.find('div', class_='collection-details__location')
    description = soup.find('div', class_='collection-details__label')
    #image = soup.findAll('img')[1]
    images = soup.findAll('img')
    #artist_test = soup.findAll('dd')[0]
    #date = soup.findAll('dd')[1]
    #medium = soup.findAll('dd')[2]
    
    

    #print 'Title: ' + title.text
    #print artist.text
    #print 'Gallery: ' + gallery.text
    #print 'Description: ' + description.text
    #print 'Image: '
    for i in images:
       if 'ng-src' in str(i):
           print i

    #print image
    #print date
    #print artist_test
    #print medium

In [ ]:
for soup in soup_objs:
    art_dict = {}
    art_dict['title'] = soup.find('h1', class_='collection-details__object-title')
    art_dict['artist'] = soup.find('div',class_='collection-details__tombstone')
    art_dict['gallery'] = soup.find('div', class_='collection-details__location')
    art_dict['description'] = soup.find('div', class_='collection-details__label')
    art_dict['image'] = soup.findAll('img')[1]
    artwork.insert_one(art_dict)
    

In [ ]:
client = MongoClient()
db = client.paintings

artwork3 = db.artwork3

#soup_objects is all your links
for i in lst: 
    for soup in soup_objs: 
        art_dict = defaultdict(list)
        dets = soup.findAll('dt', class_='collection-details__tombstone--label')
        for s in dets:
            art_dict[s.text].append(s.nextSibling.nextSibling.text)
        art_dict['title'] = soup.find('h1', class_='collection-details__object-title').encode('utf-8')
            #art_dict['artist'] = soup.find('div',class_='collection-details__tombstone').encode('utf-8')
        art_dict['gallery'] = soup.find('div', class_='collection-details__location').encode('utf-8')
        art_dict['description'] = soup.find('div', class_='collection-details__label').encode('utf-8')
        images = soup.findAll('img')
        for i in images:
            if 'ng-src' in str(i):
                art_dict['image']=i.encode('utf-8')
        artwork3.insert_one(art_dict)

In [ ]:
artwork3.count()

In [ ]:
artwork3.find_one()

In [ ]:
import pandas as pd

df = pd.DataFrame(list(artwork3.find()))

In [ ]:
df.head()

In [ ]:
response = requests.get(test)
page = response.text
soup = BeautifulSoup(page)

In [ ]:
print art_dict

In [ ]:
#julia's notes

from collections import defaultdict

soup_objs = []

# replace my_links for test
for i in test:

In [ ]:
dets = soup_objs.findAll('dt', class_='collection-details__tombstone--label')

In [ ]:
artwork.count()

In [ ]:
artwork.find_one()

In [ ]:
db.artwork.find().limit(20)

In [ ]:
def get_links(html):
    my_links = []

    soup = BeautifulSoup(html, 'html.parser')

    for figure in soup.find_all('figure',class_="card__standard-image"):
        fig = figure.find('a')['href']
        if '/art/collection/search/' in fig:
            url = 'http://www.metmuseum.org' + fig
            my_links.append(url)
    return my_links

def get_soup_obj(my_links):
    soup_objs = []

    for i in my_links:
        response = requests.get(i)
        page = response.text
        link_soup = BeautifulSoup(page)
        soup_objs.append(link_soup)
    return soup_objs
    

def get_data(soup_objs):
    #_title = []
    #art = []
    #gall = []
    #desc = []
    #img = []
    for soup in soup_objs:
        art_dict = {}
        art_dict['title'] = soup.find('h1', class_='collection-details__object-title')
        art_dict['artist'] = soup.find('div',class_='collection-details__tombstone')
        art_dict['gallery'] = soup.find('div', class_='collection-details__location')
        art_dict['description'] = soup.find('div', class_='collection-details__label')
        art_dict['image'] = soup.findAll('img')[1]
        artwork.insert_one(art_dict)

        #_title.append(title.text)
        #art.append(artist.text)
        #gall.append(gallery.text)
        #desc.append(description.text)
        #img.append(image)
        

    #return art, gall, desc, _title, img

In [ ]:
client = MongoClient()
db = client.paintings

artwork = db.artwork

artist = []
description = []
gallery = []
image = []
title = []
for i in range(len(lst)):
    print i
    driver = webdriver.PhantomJS()  
    driver.implicitly_wait(1)
    driver.get(lst[i])
    #driver.get('http://www.metmuseum.org/art/collection#!?material=Paintings&showOnly=withImage&offset=0&pageSize=0&sortBy=Relevance&sortOrder=asc&perPage=20')
    #time.sleep(3)
    html = driver.page_source
    driver.quit()
    
    my_links = get_links(html)
    soup_obj = get_soup_obj(my_links)
    art, gall, desc, _title, img = get_data(soup_obj)
    #artist.append(art)
    #title.append(_title)
    #gallery.append(gall)
    #description.append(desc)
    #image.append(img)

In [ ]:
def get_links(html):
    my_links = []

    soup = BeautifulSoup(html, 'html.parser')

    for figure in soup.find_all('figure',class_="card__standard-image"):
        fig = figure.find('a')['href']
        if '/art/collection/search/' in fig:
            url = 'http://www.metmuseum.org' + fig
            my_links.append(url)
    return my_links

def get_soup_obj(my_links):
    soup_objs = []

    for i in my_links:
        response = requests.get(i)
        page = response.text
        link_soup = BeautifulSoup(page)
        soup_objs.append(link_soup)
    return soup_objs
    

def get_data(soup_objs):
    _title = []
    art = []
    gall = []
    desc = []
    img = []
    for soup in soup_objs:
        title = soup.find('h1', class_='collection-details__object-title')
        artist = soup.find('div',class_='collection-details__tombstone')
        gallery = soup.find('div', class_='collection-details__location')
        description = soup.find('div', class_='collection-details__label')
        image = soup.findAll('img')[1]

        _title.append(title.text)
        art.append(artist.text)
        gall.append(gallery.text)
        desc.append(description.text)
        img.append(image)
        

    return art, gall, desc, _title, img